Part 1: Setup and Initialization

This section focuses on setting up the web scraping environment and initializing the project. It involves:

~Importing the necessary libraries required for data extraction and analysis.

~Creating the project structure within a Jupyter Notebook (.ipynb) for better organization and reproducibility.

~Sending a test HTTP request to the Cars24 website to verify connectivity and ensure that the website can be accessed successfully.

This initial step is crucial as it ensures that the team members can proceed seamlessly and establish a strong foundation helping maintain efficiency, consistency, and smooth integration throughout the entire project workflow.

In [1]:
# Step 0: Dependency Installation (Optional - for new environments)
# Uncomment and run the following lines if you need to install dependencies in a new kernel
# !pip install requests beautifulsoup4 pandas

# Note: For production environments, create a requirements.txt file with:
# requests>=2.25.0
# beautifulsoup4>=4.9.0
# pandas>=1.3.0

# Then install with: pip install -r requirements.txt

print("Dependency installation cell ready (commented for safety)")

Dependency installation cell ready (commented for safety)


In [2]:
# Step 1: Importing Required Libraries and Checking Versions

import requests                      # For sending HTTP requests
from bs4 import BeautifulSoup         # For parsing HTML content
import pandas as pd                   # For data manipulation and analysis
import os                             # For creating project structure
import sys                            # For system information
import urllib.robotparser             # For robots.txt checking

# Print package versions for reproducibility
print("=== Package Versions ===")
print(f"requests: {requests.__version__}")
try:
    import bs4
    print(f"beautifulsoup4: {bs4.__version__}")
except AttributeError:
    print("beautifulsoup4: Version not available")
print(f"pandas: {pd.__version__}")
print(f"Python: {sys.version}")
print("\nLibraries imported successfully!")

=== Package Versions ===
requests: 2.32.5
beautifulsoup4: 4.14.2
pandas: 2.3.3
Python: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]

Libraries imported successfully!


In [3]:
# Step 2: Robots.txt Compliance Check
# Demonstrating awareness of scraping rules and ethical guidelines

def check_robots_txt(base_url):
    """Check robots.txt for scraping permissions"""
    try:
        rp = urllib.robotparser.RobotFileParser()
        rp.set_url(f"{base_url}/robots.txt")
        rp.read()
        
        print("=== Robots.txt Analysis ===")
        print(f"Checking robots.txt for: {base_url}")
        
        # Check if we can access the main page
        can_fetch = rp.can_fetch("*", "/")
        print(f"Can fetch main page: {can_fetch}")
        
        # Check specific paths we might need
        search_paths = ["/buy-used-hyundai-cars-mumbai/", "/buy-used-cars/"]
        for path in search_paths:
            can_access = rp.can_fetch("*", path)
            print(f"Can access {path}: {can_access}")
        
        # Get crawl delay if specified
        crawl_delay = rp.crawl_delay("*")
        if crawl_delay:
            print(f"Recommended crawl delay: {crawl_delay} seconds")
        else:
            print("No specific crawl delay specified")
            
        return rp
        
    except Exception as e:
        print(f"Error checking robots.txt: {e}")
        return None

# Check robots.txt for Cars24
base_url = "https://www.cars24.com"
robots_parser = check_robots_txt(base_url)


=== Robots.txt Analysis ===
Checking robots.txt for: https://www.cars24.com
Can fetch main page: False
Can access /buy-used-hyundai-cars-mumbai/: False
Can access /buy-used-cars/: False
No specific crawl delay specified


In [4]:
# Step 3: Basic HTTP Connectivity Test
# Simple connectivity check without detailed exception handling

# Create a session with proper headers
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
})

# Make request
response = session.get("https://www.cars24.com/buy-used-hyundai-cars-mumbai/?sort=bestmatch&serveWarrantyCount=true&listingSource=Homepage_Filters", timeout=10)

print("=== Website Connectivity Test ===")
print(f"✓ HTTP Status: {response.status_code}")
print(f"✓ Content Length: {len(response.content)} bytes")
print(f"✓ Content Type: {response.headers.get('content-type', 'Unknown')}")

# Basic page validation
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find('title')
if title:
    print(f"✓ Page Title: {title.get_text().strip()}")

print("✓ Successfully connected to Cars24 website")


=== Website Connectivity Test ===
✓ HTTP Status: 200
✓ Content Length: 1241187 bytes
✓ Content Type: text/html; charset=utf-8
✓ Page Title: 436 Hyundai Used Cars in Mumbai | Second Hand Hyundai Cars in Mumbai starting from ₹0.89 lakh - CARS24
✓ Successfully connected to Cars24 website


In [5]:
# Step 4: Project Structure Setup

project_dir = "cars24_hyundai_mumbai"              # name of the project folder
if not os.path.exists(project_dir):                # Check if the directory already exists
    os.makedirs(project_dir)                       # If not, create the directory
    print(f"Project directory '{project_dir}' created successfully.")
else:
    print(f"Project directory '{project_dir}' already exists.")                # Printing a message if it already exists

Project directory 'cars24_hyundai_mumbai' created successfully.


Part 2 – Website Declaration and Exception Handling

In [13]:
# Step 5: Exception Handling 

# Declaring the target website URL
# Using the URL tested by Part 1 Team
TARGET_URL = "https://www.cars24.com/buy-used-hyundai-cars-mumbai/?sort=bestmatch&serveWarrantyCount=true&listingSource=Homepage_Filters"

# Necessary requests exceptions are imported for precise handling
from requests.exceptions import Timeout, ConnectionError, HTTPError, RequestException
import requests

def get_robust_response(url: str, session: requests.Session, timeout_seconds: int = 15) -> requests.Response or None:
    """
    Builds a robust function to send an HTTP GET request and handle common exceptions.

    Args:
        url (str): The target website URL.
        session (requests.Session): The pre-configured requests session object.
        timeout_seconds (int): The maximum time to wait for a response.

    Returns:
        requests.Response or None: The response object if successful (status 200), otherwise None.
    """
    
    #Using try-except blocks and print descriptive error messages
    try:
        print(f"\n--- Attempting to fetch: {url} ---")
        
        # Make the request using the provided session and timeout
        response = session.get(url, timeout=timeout_seconds)
        
        # Handle Invalid Responses (for status codes 4xx/5xx)
        # This will raise an HTTPError for bad status codes
        response.raise_for_status() 
        
        print(f" Success! Status Code: {response.status_code}. Content length: {len(response.content)} bytes.")
        return response

    except Timeout:
        # Handling Timeout errors
        print(f" ERROR: Request timed out after {timeout_seconds}s for URL: {url}.")
        print(" Suggestion: Check network stability or increase the timeout limit.")
        return None
    
    except ConnectionError as e:
        # Handling Connection errors
        print(f" ERROR: Connection error occurred for URL: {url}.")
        print(f"   Details: {e.__class__.__name__} - Could not establish a connection (DNS failure, refusal, etc.).")
        return None
        
    except HTTPError as e:
        # Handling Invalid Responses (4xx or 5xx status codes)
        print(f" ERROR: Invalid HTTP Response received for URL: {url}.")
        print(f"   Details: {e.response.status_code} {e.response.reason} - The server rejected the request.")
        return None
        
    except RequestException as e:
        # Catch-all for any other requests-related error
        print(f" ERROR: An unexpected RequestException occurred for URL: {url}.")
        print(f"   Details: {e.__class__.__name__} - {e}")
        return None
    
    except Exception as e:
        # Catch-all for any non-requests related error (e.g., programming error)
        print(f" FATAL ERROR: An unexpected error occurred.")
        print(f"   Details: {e.__class__.__name__} - {e}")
        return None




In [14]:
# Part 2 Testing: Verifying the robust function

print("=== Part 2: Robust Function Testing ===")

# Reuse the session created in Part 1/Step 3
if 'session' not in locals():
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    })

# Test Case 1: Successful Connection (Expected)
print("\n[Test 1: Successful URL]")
response_success = get_robust_response(TARGET_URL, session)
if response_success:
    print("Test 1 Result: Response object successfully returned.")

# Test Case 2: 404 Not Found (Expected to fail gracefully)
print("\n[Test 2: Invalid/404 URL]")
response_404 = get_robust_response("https://www.cars24.com/this-path-does-not-exist-12345/", session)
if not response_404:
    print("Test 2 Result: Function correctly returned None after HTTPError.")

=== Part 2: Robust Function Testing ===

[Test 1: Successful URL]

--- Attempting to fetch: https://www.cars24.com/buy-used-hyundai-cars-mumbai/?sort=bestmatch&serveWarrantyCount=true&listingSource=Homepage_Filters ---
 Success! Status Code: 200. Content length: 1241376 bytes.
Test 1 Result: Response object successfully returned.

[Test 2: Invalid/404 URL]

--- Attempting to fetch: https://www.cars24.com/this-path-does-not-exist-12345/ ---
 ERROR: Invalid HTTP Response received for URL: https://www.cars24.com/this-path-does-not-exist-12345/.
   Details: 404 Not Found - The server rejected the request.
Test 2 Result: Function correctly returned None after HTTPError.


In [7]:
# Step 6: Data Cleaning
# To be completed by the next team members
